<img src="img/logo_demcompare.png" width="100" align="right">

# Demcompare: statistics tutorial with all metrics

This notebook presents how to use all the metrics defined in demcompare.
The goal is to compare two DEMs together.

## Imports and external functions

In [ ]:
import pyproj # pyproj as first import is necessary

In [ ]:
from snippets.utils_notebook import *

In [ ]:
from IPython.display import HTML, display
import tabulate

### 1. Comparing two DEMs

We need to load the two DEMs.

In [ ]:
from demcompare.dem_tools import load_dem

In [ ]:
input_ref_config = {
            "path" : "data/grenoble/Copernicus_DSM_10_N45_00_E005_00_DEM.tif",
            "zunit" : "m",
            "georef": "WGS84",
            "classification_layers": {
                "Status": {"map_path": "data/grenoble/copernicus_status.tif"}
            }
        }
input_sec_config = {
            "path" : "data/grenoble/Copernicus_blurred_and_shifted.tif",
            "zunit" : "m",
            "georef": "EPSG:32630",
            "nodata" : -32768,
        }

Loading the DEMs

In [ ]:
input_ref = load_dem(
    path=input_ref_config["path"], 
    zunit=input_ref_config["zunit"],
    classification_layers=input_ref_config["classification_layers"]
)

input_sec = load_dem(
    path=input_sec_config["path"], 
    zunit=input_sec_config["zunit"], 
)

We can see that there are differences in terms of size and resolution. But there is also an offset between them.

In [ ]:
show(side_by_side_fig(input_ref, 
                 input_sec,
                "Reference DEM", 
                "Second DEM"))

DEMs must have same size and resolution so we need to reproject them. 

In [ ]:
from demcompare.dem_tools import reproject_dems
reproj_sec, reproj_ref, _ = reproject_dems(input_sec, input_ref, sampling_source = "ref")

Computing the slope for both DEMs with `compute_dem_slope`. The slope will be used to compute metrics.

In [ ]:
from demcompare.dem_tools import compute_dem_slope

In [ ]:
reproj_ref = compute_dem_slope(reproj_ref)
reproj_sec = compute_dem_slope(reproj_sec)

### 1.1 Comparing the two DEMs independently

In this section, we compare the 2 DEMs independently, i.e. compute statistics and produce plots for each of them indivdually

Import `DemProcessing`

In [ ]:
from demcompare.dem_processing import DemProcessing

With `DemProcessing`, compute the curvature of the two DEMs, independently

In [ ]:
dem_processing_object_ref_curvature = DemProcessing("ref-curvature")
ref_curvature = dem_processing_object_ref_curvature.process_dem(reproj_ref, reproj_sec)
dem_processing_object_sec_curvature = DemProcessing("sec-curvature")
sec_curvature = dem_processing_object_sec_curvature.process_dem(reproj_ref, reproj_sec)

Show the curvature of the two DEMs side-by-side

In [ ]:
show(side_by_side_fig(ref_curvature, 
                sec_curvature,
                "Reference DEM Curvature", 
                "Second DEM Curvature"))

Create a dictonnary with a `statistics` section, and the `global` classification layer inside.
We won't use other classifications layers here, but it is possible.

In [ ]:
cfg = {
    "statistics": {
        "global": {
        }
    }
}

Import `StatsProcessing` in order to be able to produce statistics.

In [ ]:
from demcompare.stats_processing import StatsProcessing

Create a `stats_processing_` object for the two DEMs independently.

In [ ]:
stats_processing_ref = StatsProcessing(cfg['statistics'], reproj_ref)
stats_processing_sec = StatsProcessing(cfg['statistics'], reproj_sec)

Compute statistics for the two DEMs independently.
The statistics computed here are:
- `slope-orientation-histogram`
- `hillshade`
- `svf`

In [ ]:
%%capture
stats_dataset_ref = stats_processing_ref.compute_stats(metrics = ["slope-orientation-histogram", "hillshade", "svf"])
stats_dataset_sec = stats_processing_sec.compute_stats(metrics = ["slope-orientation-histogram", "hillshade", "svf"])

Get the `slope_orientation_histogram`s for the two DEMs independently

In [ ]:
slope_orientation_histogram_ref = stats_dataset_ref.get_classification_layer_metric(classification_layer = 'global', metric="slope-orientation-histogram", classif_class=0)
slope_orientation_histogram_sec = stats_dataset_sec.get_classification_layer_metric(classification_layer = 'global', metric="slope-orientation-histogram", classif_class=0)

Plot the `slope_orientation_histogram`s on the same figure

In [ ]:
plot_slope_orientation_histogram(slope_orientation_histogram_ref, slope_orientation_histogram_sec, "Reference DEM", "Secondary DEM")

Get the `hillshade` for the two DEMs independently

In [ ]:
hillshade_ref = stats_dataset_ref.get_classification_layer_metric(classification_layer = 'global', metric="hillshade", classif_class=0)
hillshade_sec = stats_dataset_sec.get_classification_layer_metric(classification_layer = 'global', metric="hillshade", classif_class=0)

Plot the `hillshade`s side-by-side

In [ ]:
show(side_by_side_fig(hillshade_ref, 
                hillshade_sec,
                "Reference DEM Hill shade", 
                "Second DEM Hill shade"))

Get the `svf` for the two DEMs independently

In [ ]:
svf_ref = stats_dataset_ref.get_classification_layer_metric(classification_layer = 'global', metric="svf", classif_class=0)
svf_sec = stats_dataset_sec.get_classification_layer_metric(classification_layer = 'global', metric="svf", classif_class=0)

Plot the `svf`s side-by-side

In [ ]:
show(side_by_side_fig(svf_ref, 
                svf_sec,
                "Reference DEM Sky view factor", 
                "Second DEM Sky view factor"))

### 1.2 Comparing the 2 DEMs together

In this section, we compare the 2 DEMs together, i.e. compute statistics and produce plots for their difference

With `DemProcessing`, compute the difference in altitude bewteen the two DEMs

In [ ]:
dem_processing_object_alti_diff = DemProcessing("alti-diff")
altitude_diff = dem_processing_object_alti_diff.process_dem(reproj_ref, reproj_sec)

With `DemProcessing`, compute the difference in altitude bewteen the two DEMs, and normalize it by the slope

In [ ]:
dem_processing_object_alti_diff_slope_norm = DemProcessing("alti-diff-slope-norm")
altitude_diff_slope_norm = dem_processing_object_alti_diff_slope_norm.process_dem(reproj_ref, reproj_sec)

Show the difference in altitude, and the difference in altitude normalized by the slope side-by-side

In [ ]:
show(side_by_side_fig(altitude_diff, 
                altitude_diff_slope_norm,
                "Elevation difference", 
                "Elevation difference normalized by the slope"))

With `DemProcessing`, compute the angular difference between the two DEMs

In [ ]:
dem_processing_object_angular_diff = DemProcessing("angular-diff")
angular_diff = dem_processing_object_angular_diff.process_dem(reproj_ref, reproj_sec)

Show the angular difference

In [ ]:
show_dem(angular_diff,  
         "Angular difference")